In [1]:
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:


df = pd.read_csv('income_evaluation.csv')
df.head()

NameError: name 'pd' is not defined

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              32561 non-null  int64 
 1    workclass       32561 non-null  object
 2    fnlwgt          32561 non-null  int64 
 3    education       32561 non-null  object
 4    education-num   32561 non-null  int64 
 5    marital-status  32561 non-null  object
 6    occupation      32561 non-null  object
 7    relationship    32561 non-null  object
 8    race            32561 non-null  object
 9    sex             32561 non-null  object
 10   capital-gain    32561 non-null  int64 
 11   capital-loss    32561 non-null  int64 
 12   hours-per-week  32561 non-null  int64 
 13   native-country  32561 non-null  object
 14   income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [ ]:
df.columns

Index(['age', ' workclass', ' fnlwgt', ' education', ' education-num',
       ' marital-status', ' occupation', ' relationship', ' race', ' sex',
       ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country',
       ' income'],
      dtype='object')

In [ ]:
df.columns = df.columns.str.strip()  # Remove leading/trailing spaces from column names
np.unique(df['income'])

array([' <=50K', ' >50K'], dtype=object)

In [ ]:
df.drop("fnlwgt", axis=1, inplace=True, errors='ignore')


In [ ]:
col_name=['age', ' workclass', ' education', ' education-num',
       ' marital-status', ' occupation', ' relationship', ' race', ' sex',
       ' capital-gain', ' capital-loss', ' hours-per-week', ' native-country',
       ' income']
df.colums=col_name
df.columns

C:\Users\yadav\AppData\Local\Temp\ipykernel_8156\3471829779.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.colums=col_name


Index(['age', 'workclass', 'education', 'education-num', 'marital-status',
       'occupation', 'relationship', 'race', 'sex', 'capital-gain',
       'capital-loss', 'hours-per-week', 'native-country', 'income'],
      dtype='object')

In [ ]:
df.loc[df['workclass']=='?']

Empty DataFrame
Columns: [age, workclass, education, education-num, marital-status, occupation, relationship, race, sex, capital-gain, capital-loss, hours-per-week, native-country, income]
Index: []

In [ ]:
df.loc[df['workclass']=='?','workclass'] = np.nan
df.loc[df['occupation']=='?','occupation'] = np.nan 
df.loc[df['native-country']=='?','native-country'] = np.nan

In [ ]:
df.isnull().sum()

age               0
workclass         0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [ ]:
df.dropna(inplace=True) #dropping null values

In [ ]:
#to change our income column datatype to int
df['income'] = df['income'].map({' <=50K':0, ' >50K':1})
df.head()

age          workclass   education  education-num       marital-status  \
0   39          State-gov   Bachelors             13        Never-married   
1   50   Self-emp-not-inc   Bachelors             13   Married-civ-spouse   
2   38            Private     HS-grad              9             Divorced   
3   53            Private        11th              7   Married-civ-spouse   
4   28            Private   Bachelors             13   Married-civ-spouse   

           occupation    relationship    race      sex  capital-gain  \
0        Adm-clerical   Not-in-family   White     Male          2174   
1     Exec-managerial         Husband   White     Male             0   
2   Handlers-cleaners   Not-in-family   White     Male             0   
3   Handlers-cleaners         Husband   Black     Male             0   
4      Prof-specialty            Wife   Black   Female             0   

   capital-loss  hours-per-week  native-country  income  
0             0              40   United-States       0  
1             0              13   United-States       0  
2             0              40   United-States       0  
3             0              40   United-States       0  
4             0              40            Cuba       0

In [ ]:
df.income.value_counts()

income
0    24720
1     7841
Name: count, dtype: int64

In [ ]:
#balance income column with equal number of 0 and 1
df = df.groupby('income').apply(lambda x: x.sample(df.income.value_counts().min(), replace=False)).reset_index(drop=True)
df.income.value_counts()


C:\Users\yadav\AppData\Local\Temp\ipykernel_8156\2671101363.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('income').apply(lambda x: x.sample(df.income.value_counts().min(), replace=False)).reset_index(drop=True)


income
0    7841
1    7841
Name: count, dtype: int64

In [ ]:
cat_col=df.select_dtypes(include=['object']).columns #selecting categorical columns
cat_col

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')

In [ ]:
num_col=df.select_dtypes(exclude='object') #selecting numerical columns
num_col.head()

age  education-num  capital-gain  capital-loss  hours-per-week  income
0   24              9             0             0              35       0
1   21              9             0             0              20       0
2   44              9             0             0              40       0
3   20              9             0             0              20       0
4   36              9             0             0              48       0

In [ ]:
# label encoding
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
cat_col_encoded = pd.DataFrame()

for col in cat_col:
	cat_col_encoded[col] = le.fit_transform(df[col])

cat_col_encoded

workclass  education  marital-status  occupation  relationship  race  \
0              4         11               4           8             4     4   
1              4         11               4           7             3     4   
2              4         11               2           3             0     4   
3              4         11               4           8             1     4   
4              4         11               0           3             4     4   
...          ...        ...             ...         ...           ...   ...   
15677          4          1               2           3             0     4   
15678          5          9               2          12             0     4   
15679          6         14               2          10             0     4   
15680          4          9               2           4             0     4   
15681          5         12               2          10             0     1   

       sex  native-country  
0        0              39  
1        1              39  
2        1              31  
3        0              39  
4        1              39  
...    ...             ...  
15677    1              39  
15678    1              39  
15679    1              39  
15680    1              39  
15681    1              19  

[15682 rows x 8 columns]

In [ ]:
cat_col
# or list(cat_col) if you want a Python list
label_list=['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country']

for i in label_list:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i])

In [ ]:
df.head()

age  workclass  education  education-num  marital-status  occupation  \
0   24          4         11              9               4           8   
1   21          4         11              9               4           7   
2   44          4         11              9               2           3   
3   20          4         11              9               4           8   
4   36          4         11              9               0           3   

   relationship  race  sex  capital-gain  capital-loss  hours-per-week  \
0             4     4    0             0             0              35   
1             3     4    1             0             0              20   
2             0     4    1             0             0              40   
3             1     4    0             0             0              20   
4             4     4    1             0             0              48   

   native-country  income  
0              39       0  
1              39       0  
2              31       0  
3              39       0  
4              39       0

In [ ]:
final_df=pd.concat([cat_col_encoded,num_col],axis=1)

In [ ]:
x=final_df.drop('income',axis=1)  #independent feature  
y=final_df['income']  #dependent feature

In [ ]:
#train test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
#standard scaler---normalise--range
#mean--0 standard deviation --1
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

In [ ]:
#svc
from sklearn.svm import SVC
svc=SVC()
svc.fit(x_train_scaled,y_train)


SVC()

In [ ]:
svc.score(x_test_scaled,y_test)

0.8278610137073638

In [ ]:
svc.score(x_train_scaled,y_train)

0.8276604224790753

In [ ]:
from sklearn.model_selection import GridSearchCV

#hyper parameter tuning
grid={
    "C":[0.01,0.1,1,10],
    "kernel":["linear","rbf","poly","sigmoid"],
    "degree":[1,2,5,7],
    "gamma":[0.01,1]

}
svm=SVC()
svm_cv=GridSearchCV(svm,grid,cv=5)
svm_cv.fit(x_train_scaled,y_train)